<a href="https://colab.research.google.com/github/Binal98/testrepo/blob/main/ITV_waiting_time_under_STS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Upload the file using file picker (if in Colab or Jupyter)
from google.colab import files
uploaded = files.upload()

# Read the uploaded Excel file
file_name = list(uploaded.keys())[0]  # get uploaded file name
df = pd.read_excel(file_name)

# Preview the first few rows
df.head()


Saving EcEvent_20250627_1805.xlsx to EcEvent_20250627_1805 (1).xlsx


,Timestamp,POW Name,Ops Position Id,Che Name,Move kind,Type Description,Unit Id,Work Queue,Is Twin Move?
0,25-May-28 1802,STS04,QCTP.STS04.4,EP010,LOAD,EXLN,NaN,T4Q-load-25A (40s Only),No
1,25-May-28 1803,STS06,QCTP.STS06.6,EP042,LOAD,EXLN,NaN,T4Q-load-05A (40s Only),No
2,25-May-28 1803,STS04,QCTP.STS04.4,EP022,LOAD,AVDR,MRKU2026327,T4Q-load-25A (40s Only),No
3,25-May-28 1803,STS06,QCTP.STS06.6,EP041,LOAD,AVDR,SUDU8772329,T4Q-load-05A (40s Only),No
4,25-May-28 1803,STS05,QCTP.STS05.5,EP016,LOAD,EXLN,NaN,T4Q-load-17B (40s Only),No


In [2]:
# Keep only AVDR/AVCO and EXLN records
df = df[df['Type Description'].isin(['AVDR', 'AVCO', 'EXLN'])].copy()

# Ensure the timestamp is in datetime format
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%y-%b-%d %H%M')

# Create a new column for the date
df['Date'] = df['Timestamp'].dt.date

# Sort for logical pairing
df = df.sort_values(by=['Che Name', 'Ops Position Id', 'Timestamp'])

# Preview cleaned data
df.head()


,Timestamp,POW Name,Ops Position Id,Che Name,Move kind,Type Description,Unit Id,Work Queue,Is Twin Move?,Date
29558,2025-06-20 06:18:00,STS04,QCSTACK.U,EP001,DSCH,AVCO,SEGU5678983,RFA-disch-23A (40s Only),No,2025-06-20
29559,2025-06-20 06:18:00,STS04,QCSTACK.U,EP001,DSCH,AVCO,SEGU5678983,RFA-disch-23A (40s Only),No,2025-06-20
1323,2025-06-01 04:27:00,STS01,QCTP.STS01.1,EP001,DSCH,AVCO,TEMU6632113,HKN-disch-77A (40s Only),No,2025-06-01
1326,2025-06-01 04:28:00,STS01,QCTP.STS01.1,EP001,DSCH,EXLN,NaN,HKN-disch-77A (40s Only),No,2025-06-01
1436,2025-06-01 05:01:00,STS01,QCTP.STS01.1,EP001,DSCH,AVCO,TEMU6632113,HKN-disch-77A (40s Only),No,2025-06-01


In [9]:
# Prepare list to collect valid waiting pairs
records = []

# Group by Che Name (ITV), Ops Position Id, and Date
group_cols = ['Che Name', 'Ops Position Id', 'Date']
for _, group in df.groupby(group_cols):
    avdr_rows = group[group['Type Description'].isin(['AVDR', 'AVCO'])]
    exln_rows = group[group['Type Description'] == 'EXLN']

    for _, avdr in avdr_rows.iterrows():
        # Find next EXLN within 60 minutes
        matched_exln = exln_rows[
            (exln_rows['Timestamp'] > avdr['Timestamp']) &
            ((exln_rows['Timestamp'] - avdr['Timestamp']).dt.total_seconds() <= 600)
        ].head(1)

        if not matched_exln.empty:
            exln = matched_exln.iloc[0]
            wait_minutes = (exln['Timestamp'] - avdr['Timestamp']).total_seconds() / 60
            records.append({
                'Date': avdr['Date'],
                'ITV': avdr['Che Name'],
                'Ops Position': avdr['Ops Position Id'],
                'AVDR Time': avdr['Timestamp'],
                'EXLN Time': exln['Timestamp'],
                'Waiting Time (min)': round(wait_minutes, 2)
            })

# Convert to DataFrame
wait_df = pd.DataFrame(records)

# Show few matched records
wait_df.head()


,Date,ITV,Ops Position,AVDR Time,EXLN Time,Waiting Time (min)
0,2025-06-01,EP001,QCTP.STS01.1,2025-06-01 04:27:00,2025-06-01 04:28:00,1.0
1,2025-06-01,EP001,QCTP.STS01.1,2025-06-01 05:01:00,2025-06-01 05:03:00,2.0
2,2025-06-01,EP001,QCTP.STS01.1,2025-06-01 05:25:00,2025-06-01 05:27:00,2.0
3,2025-06-01,EP001,QCTP.STS01.1,2025-06-01 05:51:00,2025-06-01 05:54:00,3.0
4,2025-06-01,EP001,QCTP.STS01.1,2025-06-01 06:05:00,2025-06-01 06:07:00,2.0


In [10]:
# Group and summarize waiting times by day
daily_summary = wait_df.groupby(['Date', 'ITV', 'Ops Position']).agg(
    Count=('Waiting Time (min)', 'count'),
    Avg_Wait_Min=('Waiting Time (min)', 'mean'),
    Max_Wait_Min=('Waiting Time (min)', 'max')
).reset_index()

# Preview daily summary
daily_summary.head(10)


,Date,ITV,Ops Position,Count,Avg_Wait_Min,Max_Wait_Min
0,2025-05-28,EP001,QCTP.STS04.4,1,1.000000,1.0
1,2025-05-28,EP002,QCTP.STS01.1,3,2.333333,3.0
2,2025-05-28,EP002,QCTP.STS05.5,4,1.750000,2.0
3,2025-05-28,EP003,QCTP.STS01.1,1,3.000000,3.0
4,2025-05-28,EP003,QCTP.STS03.3,1,3.000000,3.0
5,2025-05-28,EP003,QCTP.STS05.5,2,2.000000,2.0
6,2025-05-28,EP004,QCTP.STS01.1,1,3.000000,3.0
7,2025-05-28,EP004,QCTP.STS03.3,1,4.000000,4.0
8,2025-05-28,EP004,QCTP.STS04.4,1,2.000000,2.0
9,2025-05-28,EP004,QCTP.STS05.5,2,2.500000,3.0


In [11]:
# Group by Date only and calculate overall average waiting time for the day
daily_avg_all_itvs = wait_df.groupby('Date').agg(
    Total_Events=('Waiting Time (min)', 'count'),
    Avg_Wait_All_ITVs=('Waiting Time (min)', 'mean'),
    Max_Wait_All_ITVs=('Waiting Time (min)', 'max'),
    Min_Wait_All_ITVs=('Waiting Time (min)', 'min')
).reset_index()

# Preview result
daily_avg_all_itvs.head(20)


,Date,Total_Events,Avg_Wait_All_ITVs,Max_Wait_All_ITVs,Min_Wait_All_ITVs
0,2025-05-28,178,3.387640,10.0,1.0
1,2025-05-31,361,2.736842,10.0,1.0
2,2025-06-01,1386,2.670274,10.0,1.0
3,2025-06-02,710,2.670423,10.0,1.0
4,2025-06-03,849,2.789164,10.0,1.0
5,2025-06-04,88,2.443182,9.0,1.0
6,2025-06-05,579,2.447323,10.0,1.0
7,2025-06-06,560,2.714286,10.0,1.0
8,2025-06-07,1355,2.492251,10.0,1.0
9,2025-06-08,597,2.832496,10.0,1.0


In [12]:
# Save the matched pairs
wait_df.to_excel("ITV_waiting_pairs.xlsx", index=False)

# Save the summary
daily_summary.to_excel("Daily_ITV_waiting_summary.xlsx", index=False)

print("Files saved successfully!")



Files saved successfully!


In [13]:
daily_avg_all_itvs = wait_df.groupby('Date').agg(
    Total_Events=('Waiting Time (min)', 'count'),
    Avg_Wait_All_ITVs=('Waiting Time (min)', 'mean'),
    Max_Wait_All_ITVs=('Waiting Time (min)', 'max'),
    Min_Wait_All_ITVs=('Waiting Time (min)', 'min')
).reset_index()
# Save the overall daily ITV average waiting times to Excel
daily_avg_all_itvs.to_excel("Daily_Overall_ITV_Waiting_Summary.xlsx", index=False)



In [14]:
from google.colab import files
files.download("Daily_Overall_ITV_Waiting_Summary.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>